In [21]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import os
import glob
import numpy as np
from empiricaldist import Cdf
import shutil

add_path = ''
base_path = '../simulator/bin/simData'
if add_path: base_path += f'/{add_path}'

metric_to_measure = ['pbusy', 'pfree', 'ptransmit', 'preset', 'pstable', 'pfailed', 'psent', 'epsilon']
metric_to_measure_slotframe = [*metric_to_measure, 'eb_prob', "mbr", "nbr"]

# Rank Result

In [22]:
!cd /Users/dzakyzf/lain/iotlab/simulator/bin; python3.7 compute_kpis.py --test=1

simData

 simData/ac_1_exp3_imin1_runs1_motes100/exec_numMotes_100.dat.json

 simData/riata_1_exp3_imin1_runs1_motes100/exec_numMotes_100.dat.json

 simData/qt_3_exp3_imin1_ep0.2_ad0_runs1_motes100/exec_numMotes_100.dat.json

 simData/ori_1_exp3_imin1_runs1_motes100/exec_numMotes_100.dat.json

 simData/qt_1_exp3_imin1_runs1_motes100/exec_numMotes_100.dat.json


In [23]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = pd.read_csv(f'{base_path}/comparison_merged.csv', sep='\t')
df.sort_values(by='rpljoin_time_m-mean-mean', inplace=True, ignore_index=True)
df = df[df.columns[~df.columns.str.contains('-std|dead')]]
df = df.style.highlight_min(color = 'orange', axis = 0)
df

,method,trickle_pbusy-mean-mean,current-consumed-mean-mean,rpljoin_time_m-mean-mean,last_failed_dio-sum-mean,alive_nodes-mean,ori_name
0,qt_1_exp3_imin1_runs1_motes100,0.441261,0.199508,19.847010,159.000000,99.000000,qt_1_exp3_imin1_runs1_motes100
1,ac_1_exp3_imin1_runs1_motes100,0.569372,0.196210,21.652938,175.000000,99.000000,ac_1_exp3_imin1_runs1_motes100
2,qt_3_exp3_imin1_ep0.2_ad0_runs1_motes100,0.366388,0.218103,21.774852,82.000000,99.000000,qt_3_exp3_imin1_ep0.2_ad0_runs1_motes100
3,riata_1_exp3_imin1_runs1_motes100,0.582051,0.206086,21.893788,136.000000,95.000000,riata_1_exp3_imin1_runs1_motes100
4,ori_1_exp3_imin1_runs1_motes100,0.504032,0.226923,23.542995,151.000000,96.000000,ori_1_exp3_imin1_runs1_motes100


# Convergence

In [24]:
# define the name of the directory to be deleted
fig_path = f"{base_path}/figures"

def remove_create_folder(path):
    try:
        shutil.rmtree(path)
    except OSError as e:
        print(e)

    try:
        os.makedirs(path)
    except OSError as e:
        print(e)

remove_create_folder(fig_path)

In [25]:
subfolders = list(
    [os.path.join(base_path, x) for x in os.listdir(base_path)]
)

main_result = {}

for subfolder in subfolders:
    if not os.path.isdir(subfolder):
        continue
    
    mthd = subfolder.split('/')[-1]
    for infile in glob.glob(os.path.join(subfolder, '*.dat.json')):
        with open(infile, 'r') as f:
            data = json.load(f)

            result = {}
            for k in metric_to_measure:
                new_key = f'trickle_{k}'
                result[new_key] = {}

            for k in metric_to_measure_slotframe:
                new_key = f'slotframe_{k}'
                result[new_key] = {}

            per_slotframe_run = 0
            for run_id in data.keys():
                for mote_id in data[run_id].keys():
                    if mote_id == 'global-stats':
                        per_slotframe_run = data[run_id]['global-stats']['per_slotframe_run'] 
                        result['per_slotframe_run'] = per_slotframe_run
                        continue
                
                    for it in data[run_id][mote_id]['trickle']:
                        trickle_it = int(it)
                        trickle_data = data[run_id][mote_id]['trickle'][it]

                        for k in metric_to_measure:
                            new_key = f'trickle_{k}'
                            if trickle_it not in result[new_key].keys():
                                result[new_key][trickle_it] = []
                            
                            if k in trickle_data.keys():
                                new_key = f'trickle_{k}'
                                val = trickle_data[k]
                                if val is not None:
                                    result[new_key][trickle_it].append(val)

                    for k in metric_to_measure_slotframe:
                        new_key = f'slotframe_{k}'
                        values = data[run_id][mote_id].get(f'per_slotframe_{k}', [])

                        for idx, val in enumerate(values):
                            if idx not in result[new_key].keys():
                                result[new_key][idx] = []
                            result[new_key][idx].append(val)
            algo = infile.split("/")[-2]
            main_result[algo] = result

### Figure per algorithm

In [26]:
conv_base_path = f'{fig_path}/Figure per algorithm/'
remove_create_folder(conv_base_path)

# Trickle states
for algo in sorted(main_result.keys()):
    fig = plt.figure(figsize=(15,5), dpi=300)
    
    for k in metric_to_measure:
        metric = f'trickle_{k}'

        data = []
        is_over = False
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over: plt.plot(data, label=metric, linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {algo}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Trickle_{algo}.png")
    plt.close()

# Per slotframe
for algo in sorted(main_result.keys()):
    fig = plt.figure(figsize=(15,5), dpi=300)
    per_slotframe_run = main_result[algo]['per_slotframe_run']
    
    for k in metric_to_measure_slotframe:
        metric = f'slotframe_{k}'

        data = []
        is_over = False
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            if np.max(values) > 1: is_over = True
            data.append(mean_)
        
        if not is_over:
            range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
            plt.plot(range_, data, label=metric, linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {algo}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/Slotframe_{algo}.png")
    plt.close()

[Errno 2] No such file or directory: '../simulator/bin/simData/figures/Figure per algorithm/'


### Figure per metric

In [27]:
conv_base_path = f'{fig_path}/Figure per metric/'
remove_create_folder(conv_base_path)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'trickle_{k}'

    for algo in sorted(main_result.keys()):

        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        plt.plot(data, label=f'{metric}_{algo}', linestyle='--', marker='o')
        
    plt.legend()
    plt.title(f'Trickle of {metric}')
    plt.xlabel('Trickle states')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(15,5), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in sorted(main_result.keys()):
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        range_ = [x*per_slotframe_run for x in range(1, len(data)+1)]
        plt.plot(range_, data, label=f'{metric}_{algo}', linestyle='--', marker='o')
    
    plt.legend()
    plt.title(f'Slotframe of {metric}')
    plt.xlabel('Slotframe')
    plt.ylabel('Values')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/metric_{metric}.png")
    plt.close()

[Errno 2] No such file or directory: '../simulator/bin/simData/figures/Figure per metric/'


In [28]:
# algo = 'ori_5_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'slotframe_nbr'


# data = []
# for slotframe in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][slotframe]
#     mean_ = int(np.ceil(np.mean(values)))
#     data.append(mean_)

In [29]:
# algo = 'ori_2_exp3_random3_runs1_motes10_minutes60_disprio1_autoeb3_autok1_autot1_ql1'
# metric = 'trickle_pbusy'

# data = []
# for state in sorted(main_result[algo][metric].keys()):
#     values = main_result[algo][metric][state]
#     mean_ = np.mean(values)
#     data.append(mean_)

In [30]:
# temp = np.array(data) + data[-1]
# aa = np.ceil(np.arange(5, 2, -0.1))
# data2 = [*data, *aa, *temp]
# # 
# eb_prob = 1/4
# max_ = 1 - eb_prob

In [31]:
# total = 0
# avgs = []
# avg = 0
# prev_val = None
# move = []
# move2 = []

# for idx, val in enumerate(data2):
#     total += val
#     avg = total / (idx+1)
#     avgs.append(avg)

#     m = 1
#     move.append(m)
#     eb_used_prob = eb_prob
#     eb_used_prob += (max_ / pow(2, val))
    
#     move2.append(eb_used_prob)
#     prev_val = val

# plt.plot(move2)
# plt.plot(len(move2) * [eb_prob])
# plt.show()
# plt.close()

In [32]:
# plt.plot(data2)
# plt.plot(avgs)
# plt.plot(move)
# plt.show()
# plt.close()

### CDF all algorithms

In [33]:
conv_base_path = f'{fig_path}/CDF all algorithms'
remove_create_folder(conv_base_path)

# Trickle states
for k in metric_to_measure:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'trickle_{k}'

    for algo in sorted(main_result.keys()):
        data = []
        for state in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][state]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        if len(data) > 0: Cdf.from_seq(data).plot()
        
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()


# Per slotframe
for k in metric_to_measure_slotframe:
    fig = plt.figure(figsize=(10,10), dpi=300)
    metric = f'slotframe_{k}'
    
    for algo in sorted(main_result.keys()):
        data = []
        per_slotframe_run = main_result[algo]['per_slotframe_run']
        for slotframe in sorted(main_result[algo][metric].keys()):
            values = main_result[algo][metric][slotframe]
            if not values or all(i is None for i in values): continue
            mean_ = np.mean(values)
            data.append(mean_)
        
        if len(data) > 0: Cdf.from_seq(data).plot()
    
    plt.legend(main_result.keys())
    plt.title(metric)
    plt.xlabel('Values')
    plt.ylabel('CDF')
    plt.tight_layout()
    plt.savefig(f"{conv_base_path}/{metric}_cdf.png")
    plt.close()

[Errno 2] No such file or directory: '../simulator/bin/simData/figures/CDF all algorithms'
